In [1]:
#!pip install webdriver_manager
#!pip install lxml
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
from time import sleep
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager



## PART 1: Write a function that gets the URLs of Money Diary stories

In [2]:
#Testing

r = requests.get("https://www.refinery29.com/en-us/money-diary?page=35")
parsed = bs(r.text, 'html.parser')
#This uses find_all to get everything that is <a> class (which in html references the hyperlinks)
#Then it filters the 
#results=set(filter(lambda x: re.search("R29 ?Team", x.text) is not None, parsed.find_all('a')))
#results=set(filter(lambda x: re.search("en-us", x.get('href')), parsed.find_all('a')))
#results=set(filter(lambda x: re.search("en-us.*money-diary[^?]", x.get('href')), parsed.find_all('a')))
#results=set(filter(lambda x: re.search("en-us.*money-diary[^extras]", x.get('href')), parsed.find_all('a')))

results=set(filter(lambda x: "extras" not in x.get('href') and "money-diar" in x.get('href') and "?" not in x.get('href') and "signup" not in x.get('href'), parsed.find_all('a')))

#print(results)
for item in results:
    print("-----")
    #Headline text
    print(item.text)
    #URL 
    print(item.get('href'))


-----
Money DiariesA Week In Dublin, Ireland, On A $37,100 Salary"7 p.m. — We meet up ahead of time for a drink. I buy the first round, a pint of Guinness for me and an alcohol-free Paulaner for him. He gets the secondby You
/en-us/money-diary-dublin-ireland-digital-marketing-account-manager-salary
-----
Money DiariesA Week In New Orleans, LA, On A $78,000 SalaryWelcome to Money Diaries, where we're tackling what might be the last taboo facing modern working women: money. We're asking millennials how theyby You
/en-us/money-diary-new-orleans-la-controls-engineer-salary
-----
Money DiariesA Week In Ocean County, NJ, On $10/HourWelcome to Money Diaries, where we're tackling what might be the last taboo facing modern working women: money. We're asking millennials how theyby You
/en-us/money-diary-ocean-county-nj-intern-seasonal-worker-budget
-----
Money DiariesA Week In New York City On $14/Hour"One of these friends, B., texts our group and asks if anyone wants to see The Incredibles 2 to

In [3]:
#This initializes the set of links that I will eventually populate
diary_links = set()

#A.  Defines a function that, given a url (to a Money Diaries main page), gets the links of the associated stories on 
#that page.  Input is the variable url
def get_diary_links_from_url(url):
    """
    Fetches all diary links from a given url, otherwise returns None.
    """
    #goes to the url and stores the stuff on that url as the variable r
    r = requests.get(url)
    # refinery29 returns a 403 for pages that don't exist
    if r.status_code == 403:
        return None
    #Run beautiful soup -- the input is r.text (which textifies the request from the url)
    #bs(input,'html.parser' ) -- 'html.parser' is the BS thing that parses the website
    parsed = bs(r.text, 'html.parser')
    #This uses find_all to get everything that is <a> class (which in html references the hyperlinks)
    #return set(filter(lambda x: re.search("R29 ?Team", x.text), parsed.find_all('a')))
    #there is a broken link with extras in it.
    #return set(filter(lambda x: re.search("en-us.*money-diar", x.get('href')) is not None, parsed.find_all('a')))
    return set(filter(lambda x: "money-diar" in x.get('href') and "extras" not in x.get('href') and "submit" not in x.get('href') and "?" not in x.get('href') and "signup" not in x.get('href'), parsed.find_all('a')))



In [4]:

#B.  This initializes the actual running of the function we just defined
page_number = 1
MONEY_DIARY_ROOT = "https://www.refinery29.com/en-us/money-diary"
links = True
while links is not None:
    print(f"Getting links from page {page_number}")
    links = get_diary_links_from_url(f"{MONEY_DIARY_ROOT}?page={page_number}")
    if links:
        links = {l.get('href') for l in links}
        print(f"Got {len(links)} from page {page_number} ({len(links.difference(diary_links))} new)")
        diary_links.update(links)
    page_number = page_number + 1
    sleep(0.1)


Getting links from page 1
Got 34 from page 1 (34 new)
Getting links from page 2
Got 30 from page 2 (26 new)
Getting links from page 3
Got 29 from page 3 (29 new)
Getting links from page 4
Got 29 from page 4 (29 new)
Getting links from page 5
Got 30 from page 5 (30 new)
Getting links from page 6
Got 30 from page 6 (30 new)
Getting links from page 7
Got 29 from page 7 (29 new)
Getting links from page 8
Got 30 from page 8 (30 new)
Getting links from page 9
Got 30 from page 9 (30 new)
Getting links from page 10
Got 29 from page 10 (29 new)
Getting links from page 11
Got 30 from page 11 (30 new)
Getting links from page 12
Got 29 from page 12 (29 new)
Getting links from page 13
Got 29 from page 13 (29 new)
Getting links from page 14
Got 30 from page 14 (30 new)
Getting links from page 15
Got 30 from page 15 (30 new)
Getting links from page 16
Got 29 from page 16 (29 new)
Getting links from page 17
Got 29 from page 17 (29 new)
Getting links from page 18
Got 29 from page 18 (29 new)
Getting li

In [5]:
diary_link_list = list(diary_links)
print(len(diary_link_list))
#print(diary_link_list[2])
diary_links

1495


{'/en-us/teacher-buffalo-ny-salary-money-diary',
 '/en-us/money-diary-london-england-director-salary',
 '/en-us/money-diary-austin-budget',
 '/en-us/assistant-managing-editor-lawrence-kansas-salary-money-diary',
 '/en-us/money-diary-manhattan-white-plains-ny-marketing-and-events-associate-salary',
 '/en-us/money-diary-brooklyn-ny-analyst-student-salary',
 '/en-us/money-diary-chattanooga-tn-student-after-school-teacher-salary',
 '/en-us/money-diary-paris-student-budget',
 '/en-us/registered-nurse-wisconsin-salary-money-diary',
 '/en-us/unpaid-researcher-los-angeles-ca-salary-money-diary',
 '/en-us/money-diary-dallas-tx-social-media-assistant-salary',
 '/en-us/money-diary-seattle-wa-scientist-salary',
 '/en-us/it-account-executive-dallas-tx-salary-money-diary',
 '/en-us/account-manager-nyc-salary-money-diary',
 '/en-us/money-diary-dallas-vice-president-financial-planner-salary',
 '/en-us/immigration-attorney-long-beach-ca-salary-money-diary',
 '/en-us/public-health-researcher-ann-arbor-m

## PART 2A: Write function that gets the content of the links

In [6]:
#This creates a function to get the content of links
def get_editorial_content(diary_link):
    print("Getting content from url", diary_link)
    r = requests.get(diary_link)
    if r.status_code != 200:
        print(f"Invalid link: {diary_link}")
        return None
    parsed = bs(r.text, 'html.parser')
    #This is the 
    content = parsed.find_all(class_='section-text')
    if content is None:
        print(f"No .section-text(s) at {diary_link}")
    return [section.text for section in content if section.text is not None]

#This tests it out to make sure it worked
get_editorial_content(f"https://www.refinery29.com{diary_link_list[2]}")

Getting content from url https://www.refinery29.com/en-us/money-diary-austin-budget


["Welcome to Money Diaries, where we're tackling what might be the last taboo facing modern working women: money. We asked a millennial woman how she spends her hard-earned money during a seven-day period — and we're tracking every last cent. (Thanks, New York Mag, for the inspiration.) Today, a tech woman in Austin juggling a bachelorette party, a man, and a cold.",
 'Industry: TechLocation: Austin, TXSalary: $52,500Age: 26Roommates: 2\r\n\r\nMonthly Expenses:Rent, utilities, and internet: $750Student loans: $318.67Car insurance and gas: $130Cell phone: $75',
 "Day One7 a.m. — Sneak a kombucha out of my gentleman friend’s fridge and head to work, where I have my usual toast, almond butter, and coffee breakfast combo. My job provides free and surprisingly delicious coffee. I have a food hangover from a weekend filled with a potluck, tacos, and a dinner party, so I’m looking to try and keep it healthy today. \r\n\r\n8 a.m. — I’m an obsessive budgeter, and the first thing I do when I ope

## PART 2B: Get the number of comments

In [7]:
#This is done w/ selenium so we need to initialize the driver
#driver = webdriver.Chrome(ChromeDriverManager().install())

In [8]:
#Just check to make sure I can get the comments on a single link

# driver.get("https://www.refinery29.com/en-us/paralegal-brooklyn-ny-salary-money-diary")
# time.sleep(5)
# driver.execute_script("""
#     window.scrollTo({ top: document.body.scrollHeight, behavior: 'smooth' })
# """)
# time.sleep(1)
# comment_count = driver.execute_script("""
#     return document.querySelector("[data-spot-im-shadow-host*='conversation']").shadowRoot.querySelector("[data-spot-im-class='comments-count']").textContent
# """)

# comment_count


In [9]:
#This defines the function
def get_num_comments(url):
    """
    Fetches number of comments for a given url
    """
    print("Getting comments from url", url)
    driver.get(url)

    time.sleep(10)
    print("Scrolling down")
    driver.execute_script("""
    window.scrollTo({ top: document.body.scrollHeight, behavior: 'smooth' })
    """)
    time.sleep(10)

    print("Getting comment count")
    comment_count = driver.execute_script("""
    try {
        return document.querySelector("[data-spot-im-shadow-host*='conversation']").shadowRoot.querySelector("[data-spot-im-class='comments-count']").textContent
    } catch(e) {
        return null
    }
    """)
    
    return comment_count

## PART 3: Loop through all of the links to get the content and comments

In [10]:
#This is done w/ selenium so we need to initialize the driver
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======


2022-07-09 23:21:22,353 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-09 23:21:22,399 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


2022-07-09 23:21:22,400 INFO Get LATEST chromedriver version for 103.0.5060 google-chrome


[WDM] - Driver [/Users/danielleli/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache


2022-07-09 23:21:22,473 INFO Driver [/Users/danielleli/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache


/var/folders/fy/56t7kzwn4m196p75kksy9j5c0000gn/T/ipykernel_59840/3905140457.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [11]:
diary_link_list
len(diary_link_list)

1495

In [13]:
diaries=[]

#Loops over all the links to get the content 
for key, diary_link in reversed(list(enumerate(diary_link_list, 1))):
    #GET THE CONTENT AND URL FIRST
    print(f"***{key}***")
    content = get_editorial_content(f"https://www.refinery29.com{diary_link}")
    url="https://www.refinery29.com"+diary_link
    
    #GET THE NUMBER OF COMMENTS NOW
    try:
        num_comments=get_num_comments(f"https://www.refinery29.com{diary_link}")
    except Exception as e:
        print(f"Failed getting comments, {e}")
        
    diary_dict = {
        'url': url,
        'num_comments': num_comments,
        'content': content
    }
    diaries.append(diary_dict)
    sleep(2)



***1495***
Getting content from url https://www.refinery29.com/en-us/communications-manager-queens-ny-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/communications-manager-queens-ny-salary-money-diary
Scrolling down
Getting comment count
***1494***
Getting content from url https://www.refinery29.com/en-us/bahrain-ceo-producer-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/bahrain-ceo-producer-salary-money-diary
Scrolling down
Getting comment count
***1493***
Getting content from url https://www.refinery29.com/en-us/money-diary-queens-nyc-technical-program-manager-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-queens-nyc-technical-program-manager-salary
Scrolling down
Getting comment count
***1492***
Getting content from url https://www.refinery29.com/en-us/money-diary-tech-business-function-budget
Getting comments from url https://www.refinery29.com/en-us/money-diary-tech-business-function-budg

***1482***
Getting content from url https://www.refinery29.com/en-us/burbank-ca-executive-assistant-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/burbank-ca-executive-assistant-salary-money-diary
Failed getting comments, Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000100889d14 chromedriver + 3792148
1   chromedriver                        0x0000000100820828 chromedriver + 3360808
2   chromedriver                        0x00000001005210b8 chromedriver + 217272
3   chromedriver                        0x0000000100510124 chromedriver + 147748
4   chromedriver                        0x000000010050f8f4 chromedriver + 145652
5   chromedriver                        0x000000010050ef50 chromedriver + 143184
6   chromedriver                        0x000000010050e7f8 chromedriver + 141304
7   chromedriver                        0x000000010050e6e8 chromedriver + 

***1472***
Getting content from url https://www.refinery29.com/en-us/digital-operations-manager-brooklyn-ny-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/digital-operations-manager-brooklyn-ny-salary-money-diary
Failed getting comments, Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000100889d14 chromedriver + 3792148
1   chromedriver                        0x0000000100820828 chromedriver + 3360808
2   chromedriver                        0x00000001005210b8 chromedriver + 217272
3   chromedriver                        0x0000000100510124 chromedriver + 147748
4   chromedriver                        0x000000010050f8f4 chromedriver + 145652
5   chromedriver                        0x000000010050ef50 chromedriver + 143184
6   chromedriver                        0x000000010050e7f8 chromedriver + 141304
7   chromedriver                        0x000000010052769c

Scrolling down
Getting comment count
***1461***
Getting content from url https://www.refinery29.com/en-us/money-diary-glendale-ca-communications-manager-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-glendale-ca-communications-manager-salary
Scrolling down
Getting comment count
***1460***
Getting content from url https://www.refinery29.com/en-us/durham-couple-money-diaries
Getting comments from url https://www.refinery29.com/en-us/durham-couple-money-diaries
Scrolling down
Getting comment count
***1459***
Getting content from url https://www.refinery29.com/en-us/money-diary-new-york-ny-associate-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-new-york-ny-associate-salary
Scrolling down
Getting comment count
***1458***
Getting content from url https://www.refinery29.com/en-us/money-diary-somerville-ma-regulatory-affairs-manager-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-somerville-ma-regulato

Scrolling down
Getting comment count
***1444***
Getting content from url https://www.refinery29.com/en-us/contractor-washington-dc-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/contractor-washington-dc-salary-money-diary
Scrolling down
Getting comment count
***1443***
Getting content from url https://www.refinery29.com/en-us/paralegal-brooklyn-ny-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/paralegal-brooklyn-ny-salary-money-diary
Scrolling down
Getting comment count
***1442***
Getting content from url https://www.refinery29.com/en-us/associate-producer-digital-media-atlanta-ga-money-diary
Getting comments from url https://www.refinery29.com/en-us/associate-producer-digital-media-atlanta-ga-money-diary
Scrolling down
Getting comment count
***1441***
Getting content from url https://www.refinery29.com/en-us/software-developer-new-zealand-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/software

...

***25***
Getting content from url https://www.refinery29.com/en-us/money-diary-brooklyn-ny-coordinator-of-family-services-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-brooklyn-ny-coordinator-of-family-services-salary
Failed getting comments, Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000100889d14 chromedriver + 3792148
1   chromedriver                        0x0000000100820828 chromedriver + 3360808
2   chromedriver                        0x00000001005210b8 chromedriver + 217272
3   chromedriver                        0x0000000100510124 chromedriver + 147748
4   chromedriver                        0x000000010050f8f4 chromedriver + 145652
5   chromedriver                        0x000000010050ef50 chromedriver + 143184
6   chromedriver                        0x000000010050e7f8 chromedriver + 141304
7   chromedriver                        0x0000000100

***20***
Getting content from url https://www.refinery29.com/en-us/money-diary-oakland-ca-elementary-afterschool-director-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-oakland-ca-elementary-afterschool-director-salary
Failed getting comments, Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000100889d14 chromedriver + 3792148
1   chromedriver                        0x0000000100820828 chromedriver + 3360808
2   chromedriver                        0x00000001005210b8 chromedriver + 217272
3   chromedriver                        0x0000000100510124 chromedriver + 147748
4   chromedriver                        0x000000010050f8f4 chromedriver + 145652
5   chromedriver                        0x000000010050ef50 chromedriver + 143184
6   chromedriver                        0x000000010050e7f8 chromedriver + 141304
7   chromedriver                        0x0000000100

***15***
Getting content from url https://www.refinery29.com/en-us/money-diary-dallas-vice-president-financial-planner-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-dallas-vice-president-financial-planner-salary
Failed getting comments, Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000100889d14 chromedriver + 3792148
1   chromedriver                        0x0000000100820828 chromedriver + 3360808
2   chromedriver                        0x00000001005210b8 chromedriver + 217272
3   chromedriver                        0x0000000100510124 chromedriver + 147748
4   chromedriver                        0x000000010050f8f4 chromedriver + 145652
5   chromedriver                        0x000000010050ef50 chromedriver + 143184
6   chromedriver                        0x000000010050e7f8 chromedriver + 141304
7   chromedriver                        0x000000010050e6e8

***10***
Getting content from url https://www.refinery29.com/en-us/unpaid-researcher-los-angeles-ca-salary-money-diary
Getting comments from url https://www.refinery29.com/en-us/unpaid-researcher-los-angeles-ca-salary-money-diary
Failed getting comments, Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000100889d14 chromedriver + 3792148
1   chromedriver                        0x0000000100820828 chromedriver + 3360808
2   chromedriver                        0x00000001005210b8 chromedriver + 217272
3   chromedriver                        0x0000000100510124 chromedriver + 147748
4   chromedriver                        0x000000010050f8f4 chromedriver + 145652
5   chromedriver                        0x000000010050ef50 chromedriver + 143184
6   chromedriver                        0x000000010050e7f8 chromedriver + 141304
7   chromedriver                        0x000000010050e6e8 chromedriver 

***5***
Getting content from url https://www.refinery29.com/en-us/money-diary-manhattan-white-plains-ny-marketing-and-events-associate-salary
Getting comments from url https://www.refinery29.com/en-us/money-diary-manhattan-white-plains-ny-marketing-and-events-associate-salary
Failed getting comments, Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000100889d14 chromedriver + 3792148
1   chromedriver                        0x0000000100820828 chromedriver + 3360808
2   chromedriver                        0x00000001005210b8 chromedriver + 217272
3   chromedriver                        0x0000000100510124 chromedriver + 147748
4   chromedriver                        0x000000010050f8f4 chromedriver + 145652
5   chromedriver                        0x000000010050ef50 chromedriver + 143184
6   chromedriver                        0x000000010050e7f8 chromedriver + 141304
7   chromedriver         

## PART 4: Put everything into a dataframe

In [14]:
df=pd.DataFrame(diaries)
df.to_csv('moneydiariesC.csv', index=False)
df.shape

(1495, 3)

In [15]:
df.head(10)

,url,num_comments,content
0,https://www.refinery29.com/en-us/communication...,312 Comments,"[Welcome to Money Diaries, where we're tacklin..."
1,https://www.refinery29.com/en-us/bahrain-ceo-p...,None,"[Welcome to Money Diaries, where we're tacklin..."
2,https://www.refinery29.com/en-us/money-diary-q...,69 Comments,"[Welcome to Money Diaries, where we're tacklin..."
3,https://www.refinery29.com/en-us/money-diary-t...,4 Comments,"[Welcome to Money Diaries, where we're tacklin..."
4,https://www.refinery29.com/en-us/austin-tx-ent...,617 Comments,"[Welcome to Money Diaries, where we're tacklin..."
5,https://www.refinery29.com/en-us/managing-dire...,154 Comments,"[Welcome to Money Diaries, where we are tackli..."
6,https://www.refinery29.com/en-us/phd-student-n...,147 Comments,[Welcome to Money Diaries where we are tacklin...
7,https://www.refinery29.com/en-us/money-diary-c...,140 Comments,"[Welcome to Money Diaries, where we're tacklin..."
8,https://www.refinery29.com/en-us/communication...,518 Comments,"[Welcome to Money Diaries, where we're tacklin..."
9,https://www.refinery29.com/en-us/money-diary-j...,4 Comments,"[Welcome to Money Diaries, where we're tacklin..."
